# Automated ML



In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from azureml.widgets import RunDetails
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Workspace & Experiment

Connect to the workspace and create an Experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'MPG-Prediction'

experiment=Experiment(ws, experiment_name)


quick-starts-ws-234780
aml-quickstarts-234780
westeurope
510b94ba-e453-4417-988b-fbdc37b55ca7


### Create or Attach an AmlCompute cluster

In [3]:

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


#### Dataset

In [4]:
dataset = Dataset.get_by_name(ws, name='mpg_pred')

#view the data
dataset.take(5).to_pandas_dataframe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
0,18.0,8,307.0,130,3504,12.0,70
1,15.0,8,350.0,165,3693,11.5,70
2,18.0,8,318.0,150,3436,11.0,70
3,16.0,8,304.0,150,3433,12.0,70
4,17.0,8,302.0,140,3449,10.5,70


In [5]:

# Split the dataset into train and test datasets
train_data, test_data = dataset.random_split(percentage=0.8, seed=10)

label = "mpg"

## AutoML Configuration

Here the automl settings are choosen with and a regression task is passed to the automl config

In [6]:
# automl settings:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "r2_score",
    "enable_early_stopping": True,
    "max_concurrent_iterations": 5,
    "experiment_timeout_minutes": 30
   
}

# automl config parameters:
automl_config = AutoMLConfig(
    task="regression",
    compute_target=compute_target,
    training_data=train_data,
    label_column_name=label,
    **automl_settings,
)


In [7]:
# Submit your experiment

remote_run = experiment.submit(automl_config, show_output= True)


Submitting remote run.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml


Experiment,Id,Type,Status,Details Page,Docs Page
MPG-Prediction,AutoML_627bbbc0-c27d-4488-b9ed-663a5abaa043,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Generating individually featurized CV splits.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+------------------------------+------------------------------+------------------------------+
|Column name                   |Missing value count  

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In our case the Voting Ensamble Model performed the best compared to other models as it had a better R2 Score as the model fit the data better than other models. There could be varaiety of factors that can affect performance such as batch size, learning rate, dropout,regularization methods, etc which can effect the performace.

There could be other factors such as variance and corelation in the dataset which effects some models but other models have weights and techniques to get by it delivering better performance. 



In [8]:
#monitor run details

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion()

{'runId': 'AutoML_627bbbc0-c27d-4488-b9ed-663a5abaa043',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2023-06-01T02:45:38.526203Z',
 'endTimeUtc': '2023-06-01T03:05:55.556445Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"MPG-Prediction","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-234780","workspace_name":"quick-starts-ws-234780","region":"westeurope","compute_target":"auto-ml","spark_service":nu

## Best Model




In [10]:
#Save the best model

best_run, fitted_model = remote_run.get_output()

print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

#save the model
best_run.register_model(model_name = "AutoML_best_run", model_path = 'outputs/')



Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

best run details : {'runId': 'AutoML_627bbbc0-c27d-4488-b9ed-663a5abaa043_38', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2023-06-01T03:04:55.465708Z', 'endTimeUtc': '2023-06-01T03:05:54.107605Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'r2_score\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'MPG-Prediction\',\'compute_target\':\'auto-ml\',\'subscription_id\':\'510b94ba-e453-4417-988b-fbdc37b55ca7\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_627bbbc0-c27d-4488-b9ed-663a5abaa043_38","experiment_name":"MPG-Prediction","workspace_name":"quick-starts-ws-234780","subscription_id":"510b94ba-e453-4

Model(workspace=Workspace.create(name='quick-starts-ws-234780', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-234780'), name=AutoML_best_run, id=AutoML_best_run:1, version=1, tags={}, properties={})